# Introduction

This is a brief introduction to using the refractivesqlite package. 

This package allows to create a SQLite database from the database refractiveindex.info, interpolate the refractive indices expressed as formulas, searching, and exporting materials to CSV.

# Create the database

Note: once created, you don't need to create the DB again!
In both methods you can specify the interpolation_points for the case when *n* is expressed as a formula and *k* is not defined.

## From url

In [ ]:
from refractivesqlite import dboperations as DB

dbpath = "refractive.db"
db = DB.Database(dbpath)
db.create_database_from_url()

## Check URL version

In [ ]:
from refractivesqlite import dboperations as DB

dbpath = "refractive.db"
db = DB.Database(dbpath)
db.check_url_version()

## Load a previous version
(For example, version 2015-07-05)

In [ ]:
from refractivesqlite import dboperations as DB

dbpath = "refractive.db"
db = DB.Database(dbpath)
db.create_database_from_url(riiurl="http://refractiveindex.info/download/database/rii-database-2015-07-05.zip")

## From local yml folder

In [ ]:
from refractivesqlite import dboperations as DB

dbpath = "refractive.db"
ymlpath = "database"
db = DB.Database(dbpath)
db.create_database_from_folder(ymlpath,interpolation_points=200)

# Load database

In [4]:
from refractivesqlite import dboperations as DB

dbpath = "refractive.db"
db = DB.Database(dbpath)

Database file found at refractive.db


# Searches

# List all pages

In [ ]:
db.search_pages()

## Search pages by term

In [ ]:
db.search_pages("otanicar")

## Search pages by exact term

In [ ]:
db.search_pages("au",exact=True)

## Search by PageId

In [ ]:
db.search_id(1542)

## Search materials by *n* interval

In [ ]:
db.search_n(n=0.3,delta_n=.001)

## Search materials by *k* interval

In [ ]:
db.search_k(k=0.3,delta_k=.001)

## Search materials by *n,k* range

In [ ]:
db.search_nk(n=0.3, delta_n=0.1,k=0.3,delta_k=0.1)

# Custom SQL search
You can search for custom sql queries (at your own risk). The function prints the number of results, and returns a list with n-tuples ordered by the fields on the select statement.

In [ ]:
db.search_custom('select * from pages where shelf="main" and book="Ag" and page LIKE "%k%"')

In [ ]:
db.search_custom('select wave,coeff from extcoeff where pageid = 1 and wave between 0.3 and 0.4')

In [ ]:
db.search_custom('select wave,refindex from refractiveindex where pageid = 1 and wave between 0.3 and 0.4')

In [ ]:
db.search_custom('''select p.filepath, r.wave,refindex,coeff 
                    from refractiveindex r inner join extcoeff e on r.pageid = e.pageid and r.wave = e.wave
                    inner join pages p on r.pageid = p.pageid
                    where r.wave = .301''')

## *Be careful*: Get ALL CSVs

In [ ]:
db.get_material_csv_all(outputfolder="all")

# Interact with materials

## Load material by Id

In [12]:
mat = db.get_material(1579)

Material other\soot\acetylene soot - Dalzell.yml loaded.


## Get material info

In [13]:
print(mat.get_page_info())

OrderedDict([('pageid', 1579), ('shelf', '3d'), ('book', 'misc'), ('page', 'acetylene_soot'), ('filepath', 'other\\soot\\acetylene soot - Dalzell.yml'), ('hasrefractive', 1), ('hasextinction', 1), ('rangeMin', 0.4358), ('rangeMax', 10.0), ('points', 13)])


## Get material *n*

In [14]:
import numpy as np
n = mat.get_complete_refractive()
print(n)

[[0.4358, 1.56], [0.45, 1.56], [0.55, 1.56], [0.65, 1.57], [0.8065, 1.57], [2.5, 2.31], [3.0, 2.62], [4.0, 2.74], [5.0, 2.88], [6.0, 3.22], [7.0, 3.49], [8.5, 4.22], [10.0, 4.8]]


## Get material *n* as numpy array

In [15]:
import numpy as np
n = np.array(mat.get_complete_refractive())
print(n)

[[  0.4358   1.56  ]
 [  0.45     1.56  ]
 [  0.55     1.56  ]
 [  0.65     1.57  ]
 [  0.8065   1.57  ]
 [  2.5      2.31  ]
 [  3.       2.62  ]
 [  4.       2.74  ]
 [  5.       2.88  ]
 [  6.       3.22  ]
 [  7.       3.49  ]
 [  8.5      4.22  ]
 [ 10.       4.8   ]]


## Get material *k*

In [16]:
k = mat.get_complete_extinction()
print(k)

[[0.4358, 0.46], [0.45, 0.48], [0.55, 0.46], [0.65, 0.44], [0.8065, 0.46], [2.5, 1.26], [3.0, 1.62], [4.0, 1.64], [5.0, 1.82], [6.0, 1.84], [7.0, 2.17], [8.5, 3.46], [10.0, 3.82]]


## Get material *k*

In [17]:
k = np.array(mat.get_complete_extinction())
print(k)

[[  0.4358   0.46  ]
 [  0.45     0.48  ]
 [  0.55     0.46  ]
 [  0.65     0.44  ]
 [  0.8065   0.46  ]
 [  2.5      1.26  ]
 [  3.       1.62  ]
 [  4.       1.64  ]
 [  5.       1.82  ]
 [  6.       1.84  ]
 [  7.       2.17  ]
 [  8.5      3.46  ]
 [ 10.       3.82  ]]


## Get n,k data as CSV

In [19]:
mat.to_csv(output="")

FileNotFoundError: [Errno 2] No such file or directory: ''